In [ ]:
import pandas as pd
import numpy as np
import cv2
from PIL import Image, ImageDraw, ImageFilter, ImageOps
import random
import os
import auto_maskmix_util
import matplotlib.pyplot as plt

In [ ]:
save_dir = ""

In [ ]:
mask_cube = pd.read_csv("", names=["path", "data", "mask_cube"])
mix_img = pd.read_csv("", names=["path", "data", "mix_img"])

In [ ]:
mask_cube = mask_cube.drop(["data", "mask_cube"], axis=1)
mix_img = mix_img.drop(["data", "mix_img"], axis=1)

In [ ]:
mask_cube["head"] = mask_cube.path.str.split("/", expand=True)[8]

In [ ]:
bg_mask = mask_cube[mask_cube["head"].str.contains("mask")]

In [ ]:
bg_mask = bg_mask.sort_values("head")
bg_mask = bg_mask.reset_index(drop=True)
bg_mask

In [ ]:
background = mask_cube[mask_cube["head"].str.contains("mask") == False]

In [ ]:
background = background.sort_values("head")
background = background.reset_index(drop=True)
background

In [ ]:
mix_img = mix_img.sort_values("path")
mix_img = mix_img.reset_index(drop=True)
print(mix_img.path[0])

In [ ]:
fg_mask = pd.read_csv("/home/tamaru/scene_categorize/main/data/mask.csv", names=["path", "data", "mix_img"])
fg_mask = fg_mask.drop(["data", "mix_img"], axis=1)
fg_mask = fg_mask.sort_values("path")
fg_mask = fg_mask.reset_index(drop=True)
print(fg_mask.path[0])

In [ ]:
def auto_maskmix(FG_IMG, BG_IMG, SPEC_BG_IMG, fg_mask):
    foreground = Image.open(FG_IMG)
    background = Image.open(BG_IMG)
    foreground = foreground.resize((256,256))
    background = background.resize((256,256))
    fg_mask = Image.open(fg_mask)
#     fg_mask = ImageOps.invert(fg_mask)
    mask_im = fg_mask.resize(foreground.size).convert("L")
    
    coord = auto_maskmix_util.foreground_random_sampling(BG_IMG, SPEC_BG_IMG)
    
    back_im = background.copy()
    x, y = coord
    rate = np.random.rand()*0.5 + 0.5
    rotate = random.randint(0,360)
    foreground_ = foreground.resize((int(foreground.width*rate), int(foreground.height*rate))).rotate(rotate)
    mask_im_ = mask_im.resize((int(mask_im.width*rate), int(mask_im.height*rate))).rotate(rotate)
    #前景画像の配置位置の座標は前景画像の左上なので，中心に平行移動する->未完成
    back_im.paste(foreground_, (int(x-foreground_.size[0]/2), int(y-foreground_.size[1]/2)), mask_im_)
    print("Foreground center coordinate", int(x-foreground_.size[0]/2), int(y-foreground_.size[1]/2))
    print("Foreground Rate:", rate)
    print("Foreground Size", foreground_.size[1])
    return back_im
    

In [ ]:
def auto_maskmix_2all(background, bg_mask, mix_img, fg_mask):
    bg_col_name = background.columns.values
    bg_mask_col_name = bg_mask.columns.values
    mix_col_name = mix_img.columns.values
    background_np = np.array(background)
    bg_mask_np = np.array(bg_mask)
    mix_img_np = np.array(mix_img)
    fg_mask_np = np.array(fg_mask)
    fg_mask_col_name = fg_mask.columns.values
    i = 0
    while(background_np[i][bg_col_name.tolist().index("path")] is not None):
        rand_mix = random.randint(0,len(mix_img_np)-1)
        print("Foreground:", rand_mix)
        back_im = auto_maskmix(mix_img_np[rand_mix][0], background_np[i][bg_col_name.tolist().index("path")], bg_mask_np[i][bg_mask_col_name.tolist().index("path")], fg_mask_np[rand_mix][0])
        back_im.save(save_dir+"/"+background_np[i][bg_col_name.tolist().index("head")])
        i += 1
        print("------------")
    

In [ ]:
auto_maskmix_2all(background,bg_mask,mix_img, fg_mask)

mix_img_ = pd.read_csv("/home/tamaru/scene_categorize/main/data/mix_img.csv", names=["path", "data", "mix_img"])
mix_img_ = mix_img_.drop(["data", "mix_img"], axis=1)

def mask_with_grabcut(FG_IMG):
    bgr = cv2.imread(FG_IMG)
    bgr = cv2.cvtColor(bgr, cv2.COLOR_RGB2BGR)
    h, w = bgr.shape[:2]
    mask = np.zeros((h,w), dtype = np.uint8)
    bgdModel = np.zeros((1,65),np.float64)
    fgdModel = np.zeros((1,65),np.float64)
    rect=(1,1,w,h)
    cv2.grabCut(bgr, mask, rect, bgdModel, fgdModel, 10, cv2.GC_INIT_WITH_RECT)
    mask2 = np.where((mask==2)|(mask==0),0,1).astype('uint8')
    bgr2 = bgr*mask2[:,:,np.newaxis]
    bgr2 = cv2.resize(bgr2, (256,256))
    # 輪郭抽出する。
    contours, _ = cv2.findContours(bgr2, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # 検出された輪郭内部を255で塗りつぶす。
    mask = np.zeros_like(bgr2)
    cv2.drawContours(mask, contours, -1, color=(255,255,255), thickness=-1)

    # 以上の手順で前景画像のうち、合成する画素を255としたマスク画像が作成できた。
    mask = cv2.resize(mask, (256,256))
    return mask


mix_img_np = np.array(mix_img_)
for row in mix_img_np: 
    print(row[0])
    print(row[0].split("/")[-1])
    mask = mask_with_grabcut(row[0])
    cv2.imwrite("/home/tamaru/scene_categorize/main/data/mask/"+row[0].split("/")[-1],mask)

mix_img_np = np.array(mix_img)
for row in mix_img_np: 
    print(row[0])
    print(row[0].split("/")[-1])
    image = cv2.imread(row[0], cv2.IMREAD_GRAYSCALE)
    # 2値化する。
    _, binary = cv2.threshold(image, 10, 255, cv2.THRESH_TOZERO)
    # 輪郭抽出する。
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # 検出された輪郭内部を255で塗りつぶす。
    mask = np.zeros_like(image)
    cv2.drawContours(mask, contours, -1, color=(255,255,255), thickness=-1)
    mask =cv2.resize(mask, (256,256))
    cv2.imwrite("/home/tamaru/scene_categorize/main/data/mask/"+row[0].split("/")[-1],mask)